In [14]:
import pandas as pd
import numpy as np
import subprocess
import tempfile
import os

In [15]:
train_filepath = "train_workspace/"
animalList = ["Human","Pig","Chicken","Rat","Mouse","Dog"]
levelList = ["Strict","Relaxed","Intermediate"]
mutation_levels = ["VeryLow","Low","Medium","High"]

In [16]:
working_level = "Strict"

In [17]:
def get_df_dict(level):
    df_dict = {}    
    current_file_path = train_filepath + level + "/mutated/"
    for mutation_level in mutation_levels:                
        df_dict[mutation_level] = pd.read_pickle(current_file_path + "no-ohnologs-complete-" + mutation_level + ".pkl")        
    return df_dict

In [18]:
df_dict = get_df_dict(working_level)
df_dict["Low"].head()

,Is_Ohnolog,Is_Paralog,Sequence-1,Sequence-1 GC,Sequence-1 Id,Sequence-1 Length,Sequence-1-Transcript Id,Sequence-2,Sequence-2 GC,Sequence-2 Id,...,Evalue_Total,Bit Score_Total,Nr Hits,Sequence-1-Mutated,Sequence-1 Length-Mutated,Sequence-1 GC-Mutated,Sequence-2-Mutated,Sequence-2 Length-Mutated,Sequence-2 GC-Mutated,Mutated_Sequence_Nr
7369,0,0.0,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNCCACTTCCTCCT...,0.397366,ENSSSCG00000030602,911,ENSSSCT00000030820,GAGTGCAAAAATGCAGAGCAATAAAACTTTTAACTTGGAGAAGCAA...,0.485071,ENSSSCG00000023245,...,0.442000,23.250000,4.0,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNCCACTTCCTCCT...,910,0.401099,GAGTGCAAAAATGCAGAGCAATAAAACTTTTAACTTGGAGAAGCAA...,2311,0.485071,1
3053,0,0.0,AGCCTTGCAATCCATGCCGAGAGGAAGGCAGTGCGAGCCCGCGCCA...,0.409615,ENSG00000186479,4160,ENST00000334025,GTCCGTGGTAGGAATGCATCTTAGTTGAATATCCTCTTCAGGTTAA...,0.464370,ENSG00000157212,...,5.295714,23.828571,7.0,AGCCTTGCAATCCATGCCGAGAGGAAGGCAGTGCGAGCCCGCGCCA...,4160,0.409615,GTCCGTGGTAGGAATGCATCTTAGTTGAATATCCTCTTCAGGTTAA...,5350,0.469533,2
10223,0,0.0,AGGCGCGCTGCGGGGGAGGGGAGCGCTGATGCAAGAGGCGAGCGCG...,0.433076,ENSGALG00000033196,1427,ENSGALT00000067846,CAGCTGCAGCATGAGCAGCCCGGTGTCCGGGTCGGGCACGCAGCGG...,0.696822,ENSGALG00000043420,...,5.650909,22.081818,11.0,AGGCGCACTGCGGGGGAGGGGAGCGCTGATGCAAGAGGCGAGCGCG...,1401,0.438258,CAGCTGCAGCATGAGCAGCCCGGTGTCCGGGTCGGGCACGCAGCGG...,1227,0.696822,1
12575,0,0.0,CGGAGTCCAGACGTGCGCTGGGGCGCTCGCGGGCGCTTTCCTTCTG...,0.412182,ENSRNOG00000011367,1691,ENSRNOT00000067609,CGGGTCGGTTGTCCAAAGAGCTTGTTGAATTGTTTTTCCAGTTGTT...,0.436270,ENSRNOG00000024411,...,0.018940,33.333333,9.0,CGGAGTCCAGACGTGCGCTGGGACGCTCGCGGGCGCTTTCCTTCTG...,1690,0.418343,CGGGTCGGTTGTCCAAAGAGCTTGTTGAATTGTTTTTCCAGTTGTT...,1169,0.436270,1
1087,0,0.0,AGGCCTCCAGCTTGGTCTGTCTCCCCACCTCTACCAGCATCTGCTG...,0.526271,ENSG00000204472,1180,ENST00000466820,CGGGCGGGGCGCAGGGCAGGCTGCACCTCAGAGCGGCGGGAGCAGC...,0.674190,ENSG00000069399,...,1.983000,24.760000,10.0,AGGCCTCCAGCTTGGTCTGTCCCCACCTCTACCAGCATCTGCTCTG...,1117,0.532677,CGGGCGGGGCGCAGGGCAGGCTGCACCTCAGAGCGGCGGGAGCAGC...,2038,0.674190,1


In [19]:
def generate_fasta_file(gene_id,transcript_id,sequence,version,biotype,file):
    header = ">" + transcript_id + "." + str(version) + " "+ gene_id + ":" + transcript_id + "." + str(version) + " cdna:" + biotype + "\n"
    file.write(header + sequence)    

In [20]:
columns_name = ["Querry_Seq_Id","Subject_Seq_Id","Percent Identical Matches","Aligment Length","Nr Mismatch","Nr Gap Open","Start Align",
               "End Align","Start Align Sub","End Align Sub","Evalue","Bit Score"]
relevant_columns = ["Percent Identical Matches","Aligment Length","Nr Mismatch","Nr Gap Open","Evalue","Bit Score"]

blast_types = ["High","Medium","Low","Total"]
relevant_col_Total = [i + "_Total" for i in relevant_columns] 
relevant_col_High = [i + "_High" for i in relevant_columns]
relevant_col_Medium = [i + "_Medium" for i in relevant_columns]
relevant_col_Low = [i + "_Low" for i in relevant_columns]
total_columns = (relevant_col_High + relevant_col_Medium + relevant_col_Low + relevant_col_Total)

df_blast = {}
for mutation_level in mutation_levels:
    df = df_dict[mutation_level]    
    df_blast[mutation_level] = pd.DataFrame(columns=total_columns)
    
    list_dict_cols = {}
    list_count = []
    for colName in total_columns:
        list_dict_cols[colName] = []
        
    for index, row in df.iterrows():     
        with open("gene1.fa","w+") as f1,open("gene2.fa","w+") as f2:      
            f1.seek(0)
            f1.truncate()
            f2.seek(0)
            f2.truncate()
            generate_fasta_file(row["Sequence-1 Id"],row["Sequence-1-Transcript Id"],row["Sequence-1-Mutated"],row["Sequence-1-Transcript-Version"],row["Sequence-1-Transcript-Biotype"],f1)
            generate_fasta_file(row["Sequence-2 Id"],row["Sequence-2-Transcript Id"],row["Sequence-2-Mutated"],row["Sequence-2-Transcript-Version"],row["Sequence-2-Transcript-Biotype"],f2)                 
        list_args = ['anaconda3/envs/thesis3/bin/blastn','-query','gene1.fa','-subject','gene2.fa','-word_size','11','-outfmt','10']
        blast_querry = subprocess.Popen(list_args, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT,
        universal_newlines=True)
        stdout,stderr = blast_querry.communicate()      
        df_temp = pd.DataFrame([i.split(',') for i in stdout.split('\n') if i != ''],columns=columns_name)
        df_temp = df_temp[relevant_columns]
        
        df_temp[relevant_columns] = df_temp[relevant_columns].astype(float)
                
        matches_high = df_temp.loc[(df_temp["Evalue"]<=1e-50)]
        matches_medium = df_temp.loc[(df_temp["Evalue"]>1e-50) & (df_temp["Evalue"]<0.01)]
        matches_low = df_temp.loc[(df_temp["Evalue"]>0.01) & (df_temp["Evalue"]<10)]
              
        df_dict_level = {}
        df_dict_level["High"] = matches_high
        df_dict_level["Medium"] = matches_medium
        df_dict_level["Low"] = matches_low
        df_dict_level["Total"] = df_temp
        
        list_count.append(len(df_temp))
        
        for blast_type in blast_types:
            for relevant_column in relevant_columns:
                list_dict_cols[relevant_column + "_" + blast_type].append(df_dict_level[blast_type][relevant_column].mean())
                
    df_blast[mutation_level]=pd.DataFrame.from_dict(list_dict_cols,orient='index').transpose()
    df_blast[mutation_level]["Nr Hits"] = list_count                    
        

In [21]:
df_data_dict = {}

In [22]:
df_data_dict["VeryLow"]   = df_blast["VeryLow"].fillna(0)
df_data_dict["VeryLow"].columns = [i + "_M" for i in df_data_dict["VeryLow"].columns]
df_data_dict["VeryLow"] .head()

,Percent Identical Matches_High_M,Aligment Length_High_M,Nr Mismatch_High_M,Nr Gap Open_High_M,Evalue_High_M,Bit Score_High_M,Percent Identical Matches_Medium_M,Aligment Length_Medium_M,Nr Mismatch_Medium_M,Nr Gap Open_Medium_M,...,Nr Gap Open_Low_M,Evalue_Low_M,Bit Score_Low_M,Percent Identical Matches_Total_M,Aligment Length_Total_M,Nr Mismatch_Total_M,Nr Gap Open_Total_M,Evalue_Total_M,Bit Score_Total_M,Nr Hits_M
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.720,21.400,100.0000,11.00,0.00,0.0,0.720,21.400,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2,6.475,21.875,97.6247,12.65,0.15,0.2,6.475,21.875,20
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0000,0.00,0.00,0.0,0.000,0.000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.115,24.200,96.6665,14.00,0.50,0.0,0.115,24.200,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.230,23.300,100.0000,12.00,0.00,0.0,0.230,23.300,1


In [23]:
df_data_dict["Low"] = df_blast["Low"].fillna(0)
df_data_dict["Low"].columns = [i + "_M" for i in df_data_dict["Low"].columns]
df_data_dict["Low"].head()

,Percent Identical Matches_High_M,Aligment Length_High_M,Nr Mismatch_High_M,Nr Gap Open_High_M,Evalue_High_M,Bit Score_High_M,Percent Identical Matches_Medium_M,Aligment Length_Medium_M,Nr Mismatch_Medium_M,Nr Gap Open_Medium_M,...,Nr Gap Open_Low_M,Evalue_Low_M,Bit Score_Low_M,Percent Identical Matches_Total_M,Aligment Length_Total_M,Nr Mismatch_Total_M,Nr Gap Open_Total_M,Evalue_Total_M,Bit Score_Total_M,Nr Hits_M
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.730000,21.400000,100.000000,11.000000,0.000000,0.000000,0.730000,21.400000,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.047619,5.933333,22.033333,98.364476,12.095238,0.190476,0.047619,5.933333,22.033333,21
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,100.0,15.0,0.0,0.0,...,0.000000,0.121500,24.200000,97.777667,14.333333,0.333333,0.000000,0.082333,25.733333,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.220000,23.300000,100.000000,12.000000,0.000000,0.000000,0.220000,23.300000,1


In [24]:
df_data_dict["Medium"] = df_blast["Medium"].fillna(0)
df_data_dict["Medium"].columns = [i + "_M" for i in df_data_dict["Medium"].columns]
df_data_dict["Medium"].head()

,Percent Identical Matches_High_M,Aligment Length_High_M,Nr Mismatch_High_M,Nr Gap Open_High_M,Evalue_High_M,Bit Score_High_M,Percent Identical Matches_Medium_M,Aligment Length_Medium_M,Nr Mismatch_Medium_M,Nr Gap Open_Medium_M,...,Nr Gap Open_Low_M,Evalue_Low_M,Bit Score_Low_M,Percent Identical Matches_Total_M,Aligment Length_Total_M,Nr Mismatch_Total_M,Nr Gap Open_Total_M,Evalue_Total_M,Bit Score_Total_M,Nr Hits_M
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.054,25.10,90.000000,20.000000,1.000000,1.0,0.054000,25.10,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2,5.180,22.16,98.708300,12.200000,0.000000,0.2,5.180000,22.16,10
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0
3,0.0,0.0,0.0,0.0,0.0,0.0,100.0,15.0,0.0,0.0,...,0.0,0.420,22.35,97.777667,13.666667,0.333333,0.0,0.281333,24.50,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.220,23.30,100.000000,12.000000,0.000000,0.0,0.220000,23.30,1


In [25]:
df_data_dict["High"] = df_blast["High"].fillna(0)
df_data_dict["High"].columns = [i + "_M" for i in df_data_dict["High"].columns]
df_data_dict["High"].head()

,Percent Identical Matches_High_M,Aligment Length_High_M,Nr Mismatch_High_M,Nr Gap Open_High_M,Evalue_High_M,Bit Score_High_M,Percent Identical Matches_Medium_M,Aligment Length_Medium_M,Nr Mismatch_Medium_M,Nr Gap Open_Medium_M,...,Nr Gap Open_Low_M,Evalue_Low_M,Bit Score_Low_M,Percent Identical Matches_Total_M,Aligment Length_Total_M,Nr Mismatch_Total_M,Nr Gap Open_Total_M,Evalue_Total_M,Bit Score_Total_M,Nr Hits_M
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.570000,21.40,100.000000,11.000000,0.000000,0.0,0.570000,21.40,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.366722,22.75,98.488722,12.388889,0.222222,0.0,4.366722,22.75,18
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.052000,25.10,100.000000,13.000000,0.000000,0.0,0.052000,25.10,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.570000,21.40,100.000000,11.000000,0.000000,0.0,0.570000,21.40,2


In [26]:
for mutation_level in mutation_levels:
    df_save = pd.concat([df_dict[mutation_level].reset_index(drop=True),df_data_dict[mutation_level].reset_index(drop=True)],axis=1,sort=False)
    df_save.to_pickle(train_filepath + working_level + "/mutated/" + "no-ohnologs-complete-" + mutation_level + "2.pkl")
    
df_save.head()

,Is_Ohnolog,Is_Paralog,Sequence-1,Sequence-1 GC,Sequence-1 Id,Sequence-1 Length,Sequence-1-Transcript Id,Sequence-2,Sequence-2 GC,Sequence-2 Id,...,Nr Gap Open_Low_M,Evalue_Low_M,Bit Score_Low_M,Percent Identical Matches_Total_M,Aligment Length_Total_M,Nr Mismatch_Total_M,Nr Gap Open_Total_M,Evalue_Total_M,Bit Score_Total_M,Nr Hits_M
0,0,0.0,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNCCACTTCCTCCT...,0.397366,ENSSSCG00000030602,911,ENSSSCT00000030820,GAGTGCAAAAATGCAGAGCAATAAAACTTTTAACTTGGAGAAGCAA...,0.485071,ENSSSCG00000023245,...,0.0,0.570000,21.40,100.000000,11.000000,0.000000,0.0,0.570000,21.40,1
1,0,0.0,AGCCTTGCAATCCATGCCGAGAGGAAGGCAGTGCGAGCCCGCGCCA...,0.409615,ENSG00000186479,4160,ENST00000334025,GTCCGTGGTAGGAATGCATCTTAGTTGAATATCCTCTTCAGGTTAA...,0.464370,ENSG00000157212,...,0.0,4.366722,22.75,98.488722,12.388889,0.222222,0.0,4.366722,22.75,18
2,0,0.0,AGGCGCGCTGCGGGGGAGGGGAGCGCTGATGCAAGAGGCGAGCGCG...,0.433076,ENSGALG00000033196,1427,ENSGALT00000067846,CAGCTGCAGCATGAGCAGCCCGGTGTCCGGGTCGGGCACGCAGCGG...,0.696822,ENSGALG00000043420,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0
3,0,0.0,CGGAGTCCAGACGTGCGCTGGGGCGCTCGCGGGCGCTTTCCTTCTG...,0.412182,ENSRNOG00000011367,1691,ENSRNOT00000067609,CGGGTCGGTTGTCCAAAGAGCTTGTTGAATTGTTTTTCCAGTTGTT...,0.436270,ENSRNOG00000024411,...,0.0,0.052000,25.10,100.000000,13.000000,0.000000,0.0,0.052000,25.10,2
4,0,0.0,AGGCCTCCAGCTTGGTCTGTCTCCCCACCTCTACCAGCATCTGCTG...,0.526271,ENSG00000204472,1180,ENST00000466820,CGGGCGGGGCGCAGGGCAGGCTGCACCTCAGAGCGGCGGGAGCAGC...,0.674190,ENSG00000069399,...,0.0,0.570000,21.40,100.000000,11.000000,0.000000,0.0,0.570000,21.40,2
